In [29]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/cworthy/CT24_checkworthy_english_dev.csv
/kaggle/input/cworthy/CT24_checkworthy_english_train.csv
/kaggle/input/cworthy/CT24_checkworthy_english_dev-test.csv
/kaggle/input/cworthy/CT24_checkworthy_english/CT24_checkworthy_english/CT24_checkworthy_english_dev-test.tsv
/kaggle/input/cworthy/CT24_checkworthy_english/CT24_checkworthy_english/CT24_checkworthy_english_train.tsv
/kaggle/input/cworthy/CT24_checkworthy_english/CT24_checkworthy_english/CT24_checkworthy_english_dev.tsv
/kaggle/input/cworthytest/CT24_checkworthy_english_test.csv
/kaggle/input/cworthytest/CT24_checkworthy_english/CT24_checkworthy_english_dev.csv
/kaggle/input/cworthytest/CT24_checkworthy_english/CT24_checkworthy_english_train.csv
/kaggle/input/cworthytest/CT24_checkworthy_english/CT24_checkworthy_english_dev-test.csv
/kaggle/input/cworthytest/CT24_checkworthy_english/CT24_checkworthy_english/CT24_checkworthy_english_dev-test.tsv
/kaggle/input/cworthytest/CT24_checkworthy_english/CT24_checkworthy_engli

In [4]:
import torch
from torch.utils.data import DataLoader, TensorDataset
from transformers import BertTokenizer, BertForSequenceClassification, AdamW, get_linear_schedule_with_warmup
from sklearn.metrics import accuracy_score
import pandas as pd

# Load pre-trained BERT model and tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')
model = BertForSequenceClassification.from_pretrained('bert-base-multilingual-cased', num_labels=2)

# Load train, validation, and test datasets
train_df = pd.read_csv('/kaggle/input/cworthytest/CT24_checkworthy_arabic/CT24_checkworthy_arabic_train.csv')
val_df1 = pd.read_csv('/kaggle/input/cworthytest/CT24_checkworthy_arabic/CT24_checkworthy_arabic_dev.csv')
val_df2 = pd.read_csv('/kaggle/input/cworthytest/CT24_checkworthy_arabic/CT24_checkworthy_arabic_dev-test.csv')

val_df = pd.concat([val_df1, val_df2], ignore_index = True)

test_df = pd.read_csv('/kaggle/input/cworthytest/CT24_checkworthy_test_gold/arabic/CT24_checkworthy_arabic_test_gold.csv')

train_df = train_df.drop(['tweet_url'], axis = 1)
val_df = val_df.drop(['tweet_url'], axis = 1)
test_df = test_df.drop(['tweet_url'], axis = 1)

train_df.rename(columns = {'tweet_text':'Text'}, inplace = True)
val_df.rename(columns = {'tweet_text':'Text'}, inplace = True)
test_df.rename(columns = {'tweet_text':'Text'}, inplace = True)

test_df.head()

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at aubmindlab/bert-base-arabertv02-twitter and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


,tweet_id,Text,class_label
0,1722643241323950399,يريدون توريط السلطة الفلسطينية في الضفة و دق آ...,Yes
1,1721467098491130367,بدأت مشاهد المذبحة الكبرى بالوصول ليلة البارحة...,Yes
2,1721430495714558177,دعت دولة الإمارات والصين إلى عقد جلسة مشاورات ...,Yes
3,1721615046420086911,إسرائيل تسيطر على الإعلام الغربي، تنشر الأكاذي...,Yes
4,1721671672934129754,🚨🚨 الجزء المضاء في هذه المنطقة هو مستشفى الشفا...,Yes


In [5]:
# Preprocess the data
def preprocess_data(df):
    sentences = df['Text'].tolist()
    labels = df['class_label'].tolist()
    label_map = {'Yes': 1, 'No': 0}
    labels = [label_map[label] for label in labels]
    tokenized_texts = [tokenizer.encode(sent, add_special_tokens=True, max_length=128, truncation=True) for sent in sentences]
    max_len = max(len(tokens) for tokens in tokenized_texts)
    padded_tokenized_texts = [tokens + [0] * (max_len - len(tokens)) for tokens in tokenized_texts]
    input_ids = torch.tensor(padded_tokenized_texts)
    labels = torch.tensor(labels)
    return input_ids, labels

train_inputs, train_labels = preprocess_data(train_df)
val_inputs, val_labels = preprocess_data(val_df)
test_inputs, test_labels = preprocess_data(test_df)

# Create DataLoader for train, validation, and test datasets
batch_size = 32
train_data = TensorDataset(train_inputs, train_labels)
train_dataloader = DataLoader(train_data, batch_size=batch_size, shuffle=True)
val_data = TensorDataset(val_inputs, val_labels)
val_dataloader = DataLoader(val_data, batch_size=batch_size)
test_data = TensorDataset(test_inputs, test_labels)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

# Set optimizer and learning rate scheduler
optimizer = AdamW(model.parameters(), lr=2e-5, eps=1e-8)
epochs = 10
total_steps = len(train_dataloader) * epochs
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=total_steps)

# Train the model
model.train()
for epoch in range(epochs):
    total_loss = 0
    for batch in train_dataloader:
        batch_input_ids, batch_labels = batch
        optimizer.zero_grad()
        outputs = model(batch_input_ids, labels=batch_labels)
        loss = outputs.loss
        total_loss += loss.item()
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)  # Optional: clip gradients to prevent exploding gradients
        optimizer.step()
        scheduler.step()

    avg_train_loss = total_loss / len(train_dataloader)
    print(f'Epoch {epoch+1}/{epochs}, Average Training Loss: {avg_train_loss:.4f}')

# Evaluate on validation set
model.eval()
val_predictions = []
val_true_labels = []
for batch in val_dataloader:
    batch_input_ids, batch_labels = batch
    with torch.no_grad():
        outputs = model(batch_input_ids)
    logits = outputs.logits
    predictions = torch.argmax(logits, dim=1).tolist()
    val_predictions.extend(predictions)
    val_true_labels.extend(batch_labels.tolist())

val_accuracy = accuracy_score(val_true_labels, val_predictions)
print(f'Validation Accuracy: {val_accuracy:.4f}')

# Test the final model on test set
test_predictions = []
test_true_labels = []
for batch in test_dataloader:
    batch_input_ids, batch_labels = batch
    with torch.no_grad():
        outputs = model(batch_input_ids)
    logits = outputs.logits
    predictions = torch.argmax(logits, dim=1).tolist()
    test_predictions.extend(predictions)
    test_true_labels.extend(batch_labels.tolist())

test_accuracy = accuracy_score(test_true_labels, test_predictions)
print(f'Test Accuracy: {test_accuracy:.4f}')

/opt/conda/lib/python3.10/site-packages/transformers/optimization.py:457: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
We strongly recommend passing in an `attention_mask` since your input_ids may be padded. See https://huggingface.co/docs/transformers/troubleshooting#incorrect-output-when-padding-tokens-arent-masked.


Validation Accuracy: 0.3955
Test Accuracy: 0.4984


In [3]:
from sklearn.metrics import f1_score

# Evaluate on validation set
model.eval()
val_predictions = []
val_true_labels = []
for batch in val_dataloader:
    batch_input_ids, batch_labels = batch
    with torch.no_grad():
        outputs = model(batch_input_ids)
    logits = outputs.logits
    predictions = torch.argmax(logits, dim=1).tolist()
    val_predictions.extend(predictions)
    val_true_labels.extend(batch_labels.tolist())

# Compute F1 score for positive class
val_f1_score = f1_score(val_true_labels, val_predictions, pos_label=1)
print(f'Validation F1 Score (Positive Class): {val_f1_score:.4f}')

# Test the final model on test set
test_predictions = []
test_true_labels = []
for batch in test_dataloader:
    batch_input_ids, batch_labels = batch
    with torch.no_grad():
        outputs = model(batch_input_ids)
    logits = outputs.logits
    predictions = torch.argmax(logits, dim=1).tolist()
    test_predictions.extend(predictions)
    test_true_labels.extend(batch_labels.tolist())

# Compute F1 score for positive class
test_f1_score = f1_score(test_true_labels, test_predictions, pos_label=1)
print(f'Test F1 Score (Positive Class): {test_f1_score:.4f}')

Validation F1 Score (Positive Class): 0.0000
Test F1 Score (Positive Class): 0.0000


/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1609: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, "true nor predicted", "F-score is", len(true_sum))


In [6]:
# Test the final model on test set
test_predictions = []
test_true_labels = []
test_sentences = []
for batch in test_dataloader:
    batch_input_ids, batch_labels = batch
    with torch.no_grad():
        outputs = model(batch_input_ids)
    logits = outputs.logits
    predictions = torch.argmax(logits, dim=1).tolist()
    test_predictions.extend(predictions)
    test_true_labels.extend(batch_labels.tolist())
    test_sentences.extend([tokenizer.decode(ids, skip_special_tokens=True) for ids in batch_input_ids])

# Output class predictions of test set
results_df = pd.DataFrame({'Text': test_sentences, 'True_label': test_true_labels, 'Predicted_label': test_predictions})
results_df['Predicted_label'] = results_df['Predicted_label'].apply(lambda x: 'Yes' if x == 1 else 'No')

In [7]:
results_df.head()

,Text,True_label,Predicted_label
0,يريدون توريط السلطة الفلسطينية في الضفة و دق آ...,1,No
1,بدأت مشاهد المذبحة الكبرى بالوصول ليلة البارحة...,1,No
2,دعت دولة الإمارات والصين إلى عقد جلسة مشاورات ...,1,Yes
3,إسرائيل تسيطر على الإعلام الغربي ، تنشر الأكاذ...,1,No
4,الجزء المضاء في هذه المنطقة هو مستشفى الشفاء و...,1,No


In [8]:
test_df.head()

,tweet_id,Text,class_label
0,1722643241323950399,يريدون توريط السلطة الفلسطينية في الضفة و دق آ...,Yes
1,1721467098491130367,بدأت مشاهد المذبحة الكبرى بالوصول ليلة البارحة...,Yes
2,1721430495714558177,دعت دولة الإمارات والصين إلى عقد جلسة مشاورات ...,Yes
3,1721615046420086911,إسرائيل تسيطر على الإعلام الغربي، تنشر الأكاذي...,Yes
4,1721671672934129754,🚨🚨 الجزء المضاء في هذه المنطقة هو مستشفى الشفا...,Yes


In [10]:
results_df['Text'] = test_df[' tweet_id']

results_df.head()

,Text,True_label,Predicted_label
0,1722643241323950399,1,No
1,1721467098491130367,1,No
2,1721430495714558177,1,Yes
3,1721615046420086911,1,No
4,1721671672934129754,1,No


In [11]:
results_df = results_df.drop(['True_label'], axis = 1)
results_df.head()

,Text,Predicted_label
0,1722643241323950399,No
1,1721467098491130367,No
2,1721430495714558177,Yes
3,1721615046420086911,No
4,1721671672934129754,No


In [12]:
results_df.rename(columns = {'Predicted_label':'class_label'}, inplace = True)
results_df.rename(columns = {'Text':'id'}, inplace = True)
results_df.head()

,id,class_label
0,1722643241323950399,No
1,1721467098491130367,No
2,1721430495714558177,Yes
3,1721615046420086911,No
4,1721671672934129754,No


In [13]:
results_df['run_id'] = 'Model_1'
results_df.head()

,id,class_label,run_id
0,1722643241323950399,No,Model_1
1,1721467098491130367,No,Model_1
2,1721430495714558177,Yes,Model_1
3,1721615046420086911,No,Model_1
4,1721671672934129754,No,Model_1


In [28]:
# Output class predictions of test set to TSV
results_df.to_csv('/kaggle/working/task1_arabic.csv', index=False)
results_df.to_csv('/kaggle/working/task1_arabic.tsv', sep='\t', index=False)